In [3]:
!pip install transformers sentence-transformers langchain langchain-community langchain-openai faiss-cpu unstructured unstructured[pdf]

  Using cached openai-1.35.7-py3-none-any.whl (327 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0


In [6]:
!python freeze > requirements.txt

python3: can't open file '/content/freeze': [Errno 2] No such file or directory


In [7]:
!pip freeze > requirements.txt

In [4]:
!which python
!python --version

/usr/local/bin/python
Python 3.10.12


In [5]:
import os

from langchain_openai import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA


In [6]:
os.environ["OPENAI_API_KEY"] = ""

In [8]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [9]:
# load the document
loader = UnstructuredFileLoader("/content/hoder.pdf")
documents = loader.load()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [10]:
type(documents[0])

langchain_core.documents.base.Document

In [12]:
# create text chunks

text_splitter = CharacterTextSplitter(separator='/n',
                                      chunk_size=2000,
                                      chunk_overlap=200)

text_chunks = text_splitter.split_documents(documents)

In [13]:
# loading the vector embedding model
embeddings = HuggingFaceEmbeddings()
# vector embedding for text chunks
knowledge_base = FAISS.from_documents(text_chunks, embeddings)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [18]:
# chain for aq retrieval
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=knowledge_base.as_retriever())

In [19]:
question = "di che cosa parla questo documento?"
response = qa_chain.invoke({"query": question})
print(response["result"])

Il documento discute l'approccio diasistemico alle costruzioni multilingue, concentrandosi sull'interazione tra lingue diverse all'interno di gruppi di parlanti multilingue. Esplora come i parlanti multilingue organizzano le loro conoscenze linguistiche e come le strutture linguistiche comuni si sviluppano attraverso l'interazione tra le lingue. Viene analizzato un caso di contatto linguistico tra il latino medievale e il svedese antico, evidenziando come le categorie grammaticali e le strutture linguistiche si intersecano in un sistema comune.


In [20]:
response

{'query': 'di che cosa parla questo documento?',
 'result': "Il documento discute l'approccio diasistemico alle costruzioni multilingue, concentrandosi sull'interazione tra lingue diverse all'interno di gruppi di parlanti multilingue. Esplora come i parlanti multilingue organizzano le loro conoscenze linguistiche e come le strutture linguistiche comuni si sviluppano attraverso l'interazione tra le lingue. Viene analizzato un caso di contatto linguistico tra il latino medievale e il svedese antico, evidenziando come le categorie grammaticali e le strutture linguistiche si intersecano in un sistema comune."}

In [21]:
question = "cosa sono le diacostruzioni"
response = qa_chain.invoke({"query": question})
response["result"]

'Le diacostruzioni sono reti di costruzioni interlinguali che si formano quando i parlanti multilingue stabiliscono relazioni di equivalenza tra elementi specifici delle diverse lingue che conoscono. Queste relazioni sono basate su criteri strutturali, fonologici, semantici, funzionali o pragmatici. Le diacostruzioni rappresentano un sistema comune condiviso tra le lingue coinvolte, incorporando sia collegamenti interlinguali che elementi non specifici della lingua insieme a strutture specifiche della lingua. Le diacostruzioni sono parte di un approccio teorico alla multilinguismo e al contatto linguistico.'

In [22]:
question = "Quali sono le implicazioni per la didattica?"
response = qa_chain.invoke({"query": question})
response["result"]

'Le implicazioni per la didattica derivanti da questo studio potrebbero includere l\'importanza di considerare la multilinguismo come una caratteristica fondamentale del sistema linguistico umano. Gli insegnanti potrebbero essere incoraggiati a integrare una prospettiva multilingue nei loro approcci educativi, riconoscendo che i multilinguisti non sono semplicemente "due monolingui in uno". Inoltre, l\'idea di costruzioni interlinguali e di sistemi di diasistema potrebbe essere utilizzata per spiegare ai studenti come le lingue interagiscono e si influenzano reciprocamente. Questo potrebbe portare a una maggiore consapevolezza della complessità e della ricchezza della diversità linguistica.'

In [23]:
print(response["result"])

Le implicazioni per la didattica derivanti da questo studio potrebbero includere l'importanza di considerare la multilinguismo come una caratteristica fondamentale del sistema linguistico umano. Gli insegnanti potrebbero essere incoraggiati a integrare una prospettiva multilingue nei loro approcci educativi, riconoscendo che i multilinguisti non sono semplicemente "due monolingui in uno". Inoltre, l'idea di costruzioni interlinguali e di sistemi di diasistema potrebbe essere utilizzata per spiegare ai studenti come le lingue interagiscono e si influenzano reciprocamente. Questo potrebbe portare a una maggiore consapevolezza della complessità e della ricchezza della diversità linguistica.
